**IA & Big Data**

Prof. Miguel Bozer da Silva - miguel.bozer@senaisp.edu.br

---

In [ ]:
# Importando as bibliotecas para os modelos
import pandas as pd
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt

As tarefas 1 e 2 já foram apresentadas no arquivo 2. Label Encoding e One Hot Encoding.ipynb. Execute-os para podermos trabalhar com a divisão do conjunto de dados.

## Tarefa #1: Recebendo os dados

In [ ]:
# Recebendo os dados

carros = pd.read_csv('Used_fiat_500_in_Italy_dataset.csv', sep = ',')
carros.head()

,model,engine_power,transmission,age_in_days,km,previous_owners,lat,lon,price
0,pop,69,manual,4474,56779,2,45.071079,7.46403,4490
1,lounge,69,manual,2708,160000,1,45.069679,7.70492,4500
2,lounge,69,automatic,3470,170000,2,45.514599,9.28434,4500
3,sport,69,manual,3288,132000,2,41.903221,12.49565,4700
4,sport,69,manual,3712,124490,2,45.532661,9.03892,4790


## Tarefa #2: Corrigindo os dados

In [ ]:
carros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   model            380 non-null    object 
 1   engine_power     380 non-null    int64  
 2   transmission     380 non-null    object 
 3   age_in_days      380 non-null    int64  
 4   km               380 non-null    int64  
 5   previous_owners  380 non-null    int64  
 6   lat              380 non-null    float64
 7   lon              380 non-null    float64
 8   price            380 non-null    int64  
dtypes: float64(2), int64(5), object(2)
memory usage: 26.8+ KB


In [ ]:
carros.shape

(380, 9)

Vamos explorar as colunas que são do tipo `object` para aplicarmos o *One Hot Encoding* ou o *Label Encoding*:

In [ ]:
carros['model'].unique()

array(['pop', 'lounge', 'sport', 'star'], dtype=object)

In [ ]:
carros['transmission'].unique()

array(['manual', 'automatic'], dtype=object)

A coluna model e transmission possuem textos e precisamos corrigir isso

Vamos agora transformar a coluna de transmissão que é uma coluna que possui apenas dois valores possíveis. Para isso, vamos usando o comando o `replace`. Se o carro for manual o valor será substituído por 0 e se o carro for automático o valor será substituído por 1:

In [ ]:
carros['transmission']=carros['transmission'].map({'manual':0 , 'automatic': 1})

In [ ]:
carros['transmission'].unique()

array([0, 1])

Vamos aplicar o One Hot Enconding na coluna *model* para transformar os textos em colunas:

In [ ]:
# Aplicando o One Hot Enconding
modelos = pd.get_dummies(carros["model"], prefix = "modelo")

In [ ]:
modelos.head()

,modelo_lounge,modelo_pop,modelo_sport,modelo_star
0,False,True,False,False
1,True,False,False,False
2,True,False,False,False
3,False,False,True,False
4,False,False,True,False


Criamos dessa forma 4 colunas novas que são binárias indicando o modelo do veículo. Vamos agora criar um novo `DataFrame` unindo os `DataFrames` carros e transmissao:

In [ ]:
# Concatenando os dados:
carros_corrigidos = pd.concat([carros, modelos], axis=1)

In [ ]:
# Exibindo o novo Dataframe
carros_corrigidos.head()

,model,engine_power,transmission,age_in_days,km,previous_owners,lat,lon,price,modelo_lounge,modelo_pop,modelo_sport,modelo_star
0,pop,69,0,4474,56779,2,45.071079,7.46403,4490,False,True,False,False
1,lounge,69,0,2708,160000,1,45.069679,7.70492,4500,True,False,False,False
2,lounge,69,1,3470,170000,2,45.514599,9.28434,4500,True,False,False,False
3,sport,69,0,3288,132000,2,41.903221,12.49565,4700,False,False,True,False
4,sport,69,0,3712,124490,2,45.532661,9.03892,4790,False,False,True,False


Pensando em um modelo de *Machine Learning*, a coluna *model* pode ser excluída, pois ela não seria usada para treinar o modelo.

In [ ]:
carros_corrigidos.drop(columns=['model'], inplace = True)

In [ ]:
carros_corrigidos.head()

,engine_power,transmission,age_in_days,km,previous_owners,lat,lon,price,modelo_lounge,modelo_pop,modelo_sport,modelo_star
0,69,0,4474,56779,2,45.071079,7.46403,4490,False,True,False,False
1,69,0,2708,160000,1,45.069679,7.70492,4500,True,False,False,False
2,69,1,3470,170000,2,45.514599,9.28434,4500,True,False,False,False
3,69,0,3288,132000,2,41.903221,12.49565,4700,False,False,True,False
4,69,0,3712,124490,2,45.532661,9.03892,4790,False,False,True,False


## Tarefa #3: Dividindo o conjunto de dados

Vamos fazer a divisão do nosso conjunto de dados usando o comando `train_test_split`. Esse comando necessita dos seguintes argumentos:
* Dados de entrada;
* Dados de saída;
* test_size: Tamanho do conjunto de teste em percentual (de 0 até 1)
* random_state: opcional - Forma com a qual as linhas do conjunto de dados serão separados nos conjuntos de treinamento e de teste.


Vamos supor que desejamos criar um modelo para estimar o preço do veículo baseado nos seus outros atributos:

In [ ]:
#Dividindo os dados em Entrada/Atributos (X) e Saida/Rotulos (y)
X = carros_corrigidos.drop(columns=['price'])
y = carros_corrigidos['price']

#Divisão dos dados em treino (X_train, y_train) e teste (X_teste, y_teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Vamos ver o resultado que foi obtido para os `Dataframes` X_train, X_test, y_train e y_test:

In [ ]:
print("X_train x X_test")
print(f"{X_train.shape} x {X_test.shape}")


X_train x X_test
(266, 11) x (114, 11)


In [ ]:
X_train.columns

Index(['engine_power', 'transmission', 'age_in_days', 'km', 'previous_owners',
       'lat', 'lon', 'modelo_lounge', 'modelo_pop', 'modelo_sport',
       'modelo_star'],
      dtype='object')

In [ ]:
X_test.columns

Index(['engine_power', 'transmission', 'age_in_days', 'km', 'previous_owners',
       'lat', 'lon', 'modelo_lounge', 'modelo_pop', 'modelo_sport',
       'modelo_star'],
      dtype='object')

In [ ]:
for train, test in zip(X_train.columns, X_test.columns):
    sao_iguais = True
    if train != test:
      sao_iguais = False
if sao_iguais:
  print("Está tudo certo com as colunas")
else:
    print("O que está acontecendo?")

Está tudo certo com as colunas


Reparem que a proporção de 0.3 (30%) foi preservada entre o conjunto de teste e de treinamento:

Notem que as 11 colunas de entrada (X) são iguais para os dois conjuntos de dados, o que muda são os índices das linhas.

In [ ]:
X_train.head()

,engine_power,transmission,age_in_days,km,previous_owners,lat,lon,modelo_lounge,modelo_pop,modelo_sport,modelo_star
334,84,0,3500,94000,2,43.111359,12.38392,True,False,False,False
341,69,0,3804,87000,2,45.548222,9.18609,True,False,False,False
375,69,0,4474,55976,2,45.610050,9.24234,True,False,False,False
289,69,0,4474,55900,1,43.937881,12.71113,True,False,False,False
223,69,0,3197,139000,1,45.467960,9.18178,True,False,False,False


In [ ]:
dados ={'X_test': X_test.index,'y_test':y_test.index}
pd.DataFrame(dados)

,X_test,y_test
0,266,266
1,261,261
2,265,265
3,39,39
4,33,33
...,...,...
109,137,137
110,332,332
111,351,351
112,310,310


Para os dados de saída, reparem que os índices são iguais para X_train e y_train e X_test e y_test, mantendo a correspondencia das linhas de entrada com as linhas de saída:

In [ ]:
dados ={'X_train': X_train.index,'y_train':y_train.index}
pd.DataFrame(dados)

,X_train,y_train
0,334,334
1,341,341
2,375,375
3,289,289
4,223,223
...,...,...
261,71,71
262,106,106
263,270,270
264,348,348


# Exercício)

Para o conjunto de dados Admission_Predict.csv faça a divisão do conjunto de dados em entrada e sáida para os conjuntos de treinamento e teste com 20% dos dados no conjunto de teste. Alterne o valor do random_state. O que isso muda na divisão dos dados?